## Import repository

In [1]:
!git clone https://github.com/federico2879/MLDL2024_semantic_segmentation.git

Cloning into 'MLDL2024_semantic_segmentation'...
remote: Enumerating objects: 454, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 454 (delta 55), reused 3 (delta 3), pack-reused 349
Receiving objects: 100% (454/454), 220.69 KiB | 3.34 MiB/s, done.
Resolving deltas: 100% (262/262), done.


## Import package

In [2]:
!pip install -U fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 736.2 kB/s eta 0:00:00 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61400 sha256=d804a7aba663f3aab640e04c9cef14609025e5938e2d8928a08a9260891579e1
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31532 sha256=a3dbde91fe338d6a09b14bbf0abe0b507487f2e851f7ee110a032cf34be9d08f
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built fvcore iopath


In [3]:
import torch
from torch import nn
import torchvision
import torchvision.transforms as transforms
from torchvision.transforms.functional import InterpolationMode
from torch.utils.data import DataLoader
from MLDL2024_semantic_segmentation.datasets.cityscapes import CityScapes
from MLDL2024_semantic_segmentation.models.bisenet.build_bisenet import *
from MLDL2024_semantic_segmentation.train import *
from MLDL2024_semantic_segmentation.utils import *
from MLDL2024_semantic_segmentation.models.metrics import *
from MLDL2024_semantic_segmentation.models.IOU import *

## Setup

In [4]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

# Setup fixed parameters
num_classes = 19
num_epochs = 1

cuda


## Dataset

In [5]:
# Transformations
transform_image = transforms.Compose([
    transforms.Resize((1024, 512)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
transform_target = transforms.Compose([
    transforms.Resize((1024, 512), interpolation=InterpolationMode.NEAREST)
])

In [6]:
# Create dataloader

train_dataset = CityScapes('/kaggle/input/Cityscapes/Cityspaces', 
                           split = 'train', transform = transform_image, 
                           label_transform = transform_target)
dataloader_train = DataLoader(train_dataset, batch_size=4, shuffle=True)

val_dataset = CityScapes('/kaggle/input/Cityscapes/Cityspaces', 
                         split = 'val', transform = transform_image, 
                         label_transform = transform_target)
dataloader_val = DataLoader(val_dataset, batch_size=4, shuffle=False)

## Network, Loss, Optimizer

In [7]:
# Inizialization of the model
model = BiSeNet(num_classes=num_classes, context_path="resnet18").to(device)

#Putting on the 2 gpus
model = torch.nn.DataParallel(model, device_ids = [0,1]).to(device)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 140MB/s] 
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:01<00:00, 155MB/s]  


In [8]:
# Define loss and optimizer
loss_fn = nn.CrossEntropyLoss(ignore_index=255)
optimizer = torch.optim.SGD(model.parameters(), lr=2.5e-2,
                            momentum=0.9,weight_decay=1e-4)

## Training

In [9]:
# Set the random seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [11]:
for epoch in range(num_epochs):
    poly_lr_scheduler(optimizer, 2.5e-2, epoch, lr_decay_iter=1,
                      max_iter=num_epochs, power=0.9)
    train(model, optimizer, dataloader_train, loss_fn, num_classes, 0)
    mIOU, IOU_cl = test(model, dataloader_val, loss_fn, num_classes, 0)
    print(f"epoch: {epoch}, Validation IOU: {mIOU:.2f}")

    torch.save({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'mIOU': mIOU
    },"checkpoint.pth.tar")

print(f"Final mIOU: {mIOU:.2f}")
print("Final IOU classes")
print(IOU_cl)

flops = Flops(model, 1024, 512, device)

#print(f"Number of flops?: {flops}")

latency = Latency_FPS(model, 1024, 512, device)

print(f"Latency: {latency}")

#print(f"number of parameters: {model.count_params()}")

# Access the actual model being parallelized
actual_model = model.module
# Count the parameters of the actual model
num_params = count_params(actual_model)
print(f"number of parameters: {num_params}")

epoch: 0, Validation IOU: 0.20
Final mIOU: 0.20
Final IOU classes
[[8.90493068e-01 4.29419028e-01 6.08965234e-01 1.64922377e-02
  2.17528033e-03 9.97379512e-05 0.00000000e+00 1.24734888e-02
  6.34368924e-01 6.98503455e-02 6.20962771e-01 6.17109511e-02
  0.00000000e+00 5.09804647e-01 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 1.83922778e-02]]
| module                                      | #parameters or shape   | #flops     |
|:--------------------------------------------|:-----------------------|:-----------|
| module                                      | 12.582M                | 25.78G     |
|  saptial_path                               |  0.371M                |  5.088G    |
|   saptial_path.convblock1                   |   1.856K               |   0.243G   |
|    saptial_path.convblock1.conv1            |    1.728K              |    0.226G  |
|    saptial_path.convblock1.bn               |    0.128K              |    16.777M |
|   saptial_path.convblock2        

In [ ]:
val_mIOU,pippo = test(model, dataloader_val, loss_fn, 19, 0)
print(pippo)